# Aging son

This notebook can be used to average images of an aging kid, similar to what was done for miykael's [Noah ages](https://github.com/miykael/noah_ages) project. In short, we use the [dlib](http://dlib.net/) toolbox to detect, extract and align faces from all images. The process to do so is a 2-step process:

1. We use `hog_detector` detector to find the faces in all images. This detector is 'ok-ish' but runs very quickly.
2. For all images where it wasn't possible to detect a face, we use the `cnn_face_detection_model_v1` routine. This routine is slower but more accurate.

Dlib's face detection is usually used to extract small 'chips'/patches of pixels that only contain the face. In this case however we decided to keep the full image, but just profit from dlib's routine of aligning the faces according to the 5 landmarks (two eyes, nose and two corners of the mouth). During this procedure, images can also be rescaled to any resolution.

## Install packages

But before we can start, let's make sure that all relevant packages are installed.

# Import relevant packages

In [ ]:
from pathlib import Path
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import shutil
import pandas as pd
from skimage import io
from glob import glob
from matplotlib import patches

from tqdm.notebook import tqdm
import dlib
from PIL import Image, ImageDraw, ImageFont

# Define relevant parameters

In [ ]:
file_identifier = 'PXL_*.jpg'

In [ ]:
# Create output folder for aligned images
out_dir = Path('imgs_aligned')
if not out_dir.exists():
    out_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
# Define size of face chip
max_size = 4320
downsample = 2

# Load content

In [ ]:
# Collect filenames
path_to_img_folder = '/Users/username/Documents/photo_folder/'
path_imgs = Path(path_to_img_folder)
filenames = sorted(path_imgs.glob(file_identifier))
n_files = len(filenames)
print(f"{n_files} images were found, representing {n_files / 365.25:.2f} years.")

# Correct and align images with `skimage` and `dlib`

In [ ]:
# Load dlib models for face recognition
shape_predictor = dlib.shape_predictor('dlib/shape_predictor_5_face_landmarks.dat') # Faces landmarks (points)

# Define which face detector to use
hog_detector = dlib.get_frontal_face_detector()
cnn_detector = dlib.cnn_face_detection_model_v1('dlib/mmod_human_face_detector.dat')

### Go through with hog_detector

In [ ]:
# CNN is more advanced but takes longer; hog misses ~100 faces in total
face_detector = hog_detector

In [ ]:
# Loop through all images
for f in tqdm(filenames):

    # Specify output filename 
    out_filename = out_dir / f.name
    
    # Skip image processing if output file was already created
    if out_filename.exists():
        continue

    # Load image
    im = io.imread(f)[..., :3]

    # Get information about image size
    w, h = im.shape[:2]
    w_offset = (max_size-w)//2
    h_offset = (max_size-h)//2

    # Center image in a canvas
    canvas = np.zeros((max_size, max_size, 3)).astype('uint8')
    canvas[w_offset:w_offset+w, h_offset:h_offset+h, :] = im
    
    # Detect faces and align image
    rectangles = [x if isinstance(x, dlib.rectangle)
                  else x.rect for x in face_detector(canvas, 1)]
    if len(rectangles):
        landmarks = [shape_predictor(canvas, r) for r in rectangles]
        face_chips = [dlib.get_face_chip(
            canvas, l, size=max_size//downsample, padding=1) for l in landmarks]

        # Save aligned image
        io.imsave(out_filename, face_chips[0])

    else:
        print('No face found in', f)

**Important**: Manually delete the images in the `imgs_aligned` folder where the face detection didn't work correctly. The code below will then try to detect the face with a more advanced algorithm.

### Go through issue images with cnn_detector

In [ ]:
# CNN is more advanced but takes much longer
face_detector = cnn_detector

In [ ]:
# Loop through all images
for f in tqdm(filenames):

    # Specify output filename 
    out_filename = out_dir / f.name
    
    # Skip image processing if output file was already created
    if out_filename.exists():
        continue

    # Load image
    im = io.imread(f)[..., :3]

    # Get information about image size
    w, h = im.shape[:2]
    w_offset = (max_size-w)//2
    h_offset = (max_size-h)//2

    # Center image in a canvas
    canvas = np.zeros((max_size, max_size, 3)).astype('uint8')
    canvas[w_offset:w_offset+w, h_offset:h_offset+h, :] = im
    
    # Detect faces and align image
    rectangles = [x if isinstance(x, dlib.rectangle)
                  else x.rect for x in face_detector(canvas, 1)]
    if len(rectangles):
        landmarks = [shape_predictor(canvas, r) for r in rectangles]
        face_chips = [dlib.get_face_chip(
            canvas, l, size=max_size//downsample, padding=1) for l in landmarks]

        # Save aligned image
        io.imsave(out_filename, face_chips[0])

    else:
        print('Still no faces found in', f)

In [ ]:
# Number of images
print(f"Number of original images: {len(filenames)}")
imgs_aligned = sorted(out_dir.glob(file_identifier))
print(f"Number of aligned images:  {len(imgs_aligned)}")

**Important**: For those faces where the alignment didn't work at all, or not good enough, you will need to perform a manual alignment. The next section in this notebook will create an average image that should help to align the missing/misaligned images.

# Explorative analysis

In [ ]:
# Collect all images
imgs = np.array([io.imread(path_align)[..., :3] for path_align in tqdm(imgs_aligned)])

In [ ]:
# Create average image that can be used for alignment
img_avg = np.mean(imgs, axis=0) / 255.
img_median = np.median(imgs, axis=0) / 255.

In [ ]:
# Plot average images
fig, axs = plt.subplots(1, 2, figsize=(15, 7))
axs[0].imshow(img_avg)
axs[1].imshow(img_median)
plt.show()

In [ ]:
# Store average images to file
plt.imsave('img_01_avg.png', img_avg)
plt.imsave('img_01_median.png', img_median)

In [ ]:
# Plot a grid of certain size of first x images
grid_size = np.array([5, 3])
zoom_factor = 4
grid_points = np.prod(grid_size)
imgs_averages = np.array_split(imgs, grid_points)
fig, axes = plt.subplots(
    grid_size[1], grid_size[0], figsize=grid_size * zoom_factor, facecolor=(0, 0, 0)
)
for i, ax in zip(np.arange(grid_points), axes.flatten()):
    ax.imshow(imgs_averages[i].mean(0).astype("int"))
    ax.axis("off")
plt.tight_layout()
plt.savefig('img_02_mosaic.png')

# Video 1: No averaging

Note, get the font for the text in the image from [here](https://fonts.google.com/).

In [ ]:
# Specify frames per second
fps = 30

# Extract number of images
N_total = len(imgs_aligned)

print('Video length: %.2f seconds.' % (N_total/fps))

In [ ]:
# Save temporary folder to store images to disk
tmp_dir = Path('tmp_video_imgs')
if not tmp_dir.exists():
    tmp_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
# Collect images with text
imgs_txt = []

# Loop through all images and add text
for idx in tqdm(np.arange(len(imgs_aligned))):

    # Load aligned image
    im = io.imread(imgs_aligned[idx])

    # Collect year specific information
    n_year = idx / 365.25
    n_month = n_year * 12 % 12
    n_days = idx % 31
    n_year = int(np.floor(n_year))
    n_month = int(np.floor(n_month))
    n_days = int(np.floor(n_days))
    title_txt = f"{n_year:02d} years, {n_month+1:02d} months, {n_days+1:03d} days"

    # Establish out_filename
    out_filename = tmp_dir / f'{idx+1:04d}.jpg'

    # Add text to output filename
    img_txt = Image.fromarray(im)
    draw = ImageDraw.Draw(img_txt)
    width = max_size//downsample
    font = ImageFont.truetype("Roboto/Roboto-Light.ttf", int(width*.06))
    draw.text((width*0.1, width*0.9), title_txt, (255, 255, 255), font=font)

    # Store image to file
    io.imsave(out_filename, np.array(img_txt))
    
    # Add image with text to list
    imgs_txt.append(np.array(img_txt))

In [ ]:
# Use either code (the onze that works) to create the video
!cat tmp_video_imgs/*jpg | ffmpeg -y -f image2pipe -r $fps -vcodec mjpeg -i - -vcodec libx264 video_01_aligned.mp4

In [ ]:
# Remove all temporary files
for p in sorted(tmp_dir.glob('*jpg')):
    p.unlink()

# Create averaged images

In [ ]:
# How many images to smooth at once
smooth = 30

In [ ]:
# How many days to jump at every image
step_size = 1

In [ ]:
# Get start indeces for images
ids = [i*step_size for i in range((N_total+smooth)//step_size+1)]
len(ids)

In [ ]:
# To keep track what was already loaded
already_loaded = []
idx_range = np.arange(len(imgs_txt))

for i in tqdm(ids):
    
    # Collect indeces of images
    imgs_idx = np.arange(np.clip(i-smooth, 0, N_total-1), np.clip(i, 0, N_total-1)+1)

    # Collect images relevant for the group
    group_names = idx_range[imgs_idx]
    
    # Detect which one is new to load
    new_to_load = np.setdiff1d(group_names, already_loaded)
    
    if len(new_to_load)==0:
        pass
    elif i==0:
        imgs_group = np.array([imgs_txt[fdx] for fdx in new_to_load])
    else:
        img_new = np.array([imgs_txt[fdx] for fdx in new_to_load])
        imgs_group = np.vstack((imgs_group, img_new))
        
    # Cut imgs_group to write size
    n_offset = (i - N_total)
    if n_offset <= 0:
        n_offset = 0
    elif n_offset%2==0:
        n_offset -= 1
    imgs_group = imgs_group[-smooth+n_offset:]
    
    # Create composition image
    img_comp_mean = np.mean(imgs_group, axis=0).astype('uint8')
    img_comp_median = np.median(imgs_group, axis=0).astype('uint8')
    
    # Create out_filename
    out_filename_mean = tmp_dir / f'{i+1:04d}_mean.jpg'
    out_filename_median = tmp_dir / f'{i+1:04d}_median.jpg'
    
    # Save composition image
    io.imsave(out_filename_mean, img_comp_mean.astype('uint8'))
    io.imsave(out_filename_median, img_comp_median.astype('uint8'))

    # Keep track of what has already been loaded
    already_loaded = group_names

In [ ]:
# Use either code (the one that works) to create the video
!cat tmp_video_imgs/*mean.jpg | ffmpeg -y -f image2pipe -r $fps -vcodec mjpeg -i - -vcodec libx264 video_02_smoothed_mean_X-days.mp4
!cat tmp_video_imgs/*median.jpg | ffmpeg -y -f image2pipe -r $fps -vcodec mjpeg -i - -vcodec libx264 video_02_smoothed_median_X-days.mp4

In [ ]:
# Remove all temporary files
for p in sorted(tmp_dir.glob('*jpg')):
    p.unlink()